# pyspark3
Сделать WordCount и TopN

In [ ]:
import os
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

os.environ["HADOOP_CONF_DIR"]="/etc/hadoop/conf"
# os.environ["SPARK_HOME"]="/usr/hdp/current/spark2-client"
# os.environ["JAVA_HOME"]="/usr/java/jdk1.8.0_191/jre"

spark = pyspark.sql.SparkSession.builder\
    .master("yarn")\
    .appName("v_alehin_pyspark3")\
    .config("spark.executor.instances", "1")\
    .config("spark.executor.memory", "1G")\
    .config("spark.executor.cores", "2")\
    .config("spark.dynamicAllocation.enabled", "false")\
    .config("spark.dynamicAllocation.executorIdleTimeout", "300s")\
    .config("spark.dynamicAllocation.maxExecutors", "1000")\
    .config("spark.driver.memory", "1G")\
    .config("spark.driver.maxResultSize", "1G")\
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
    .config("spark.kryoserializer.buffer.max", "1024m")\
    .getOrCreate()

spark_context = spark.sparkContext

In [ ]:
print("""
Urls:
Yarn       http://91.219.226.252:8088/cluster/scheduler
Spark      http://91.219.226.252:8088/proxy/{app_id}/stages/
App info   http://91.219.226.252:8088/cluster/app/{app_id}/
""".format(app_id=spark_context.applicationId))

In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as sf

topn = 10

user = "v.alehin"  # your login

#raw_text_path = "/user/{}/data/data1/wctest.txt".format(user)
raw_text_path = "/user/{}/data1/books/*".format(user)

In [ ]:
docsDF = spark.read.text(raw_text_path)

In [ ]:
docsDF.printSchema()
docsDF.show(5, False)

In [ ]:
wordCountDF = (
    docsDF
    .select(
        sf.explode(
            sf.split(
                sf.regexp_replace(
                    sf.lower(sf.col("value")), 
                    "([^a-z ]+)",
                    ""),
                " ")
        ).alias("word"))
    .where(sf.col("word") != "")
    .groupBy(sf.col("word"))
    .agg(sf.count("word").alias("cnt"))
)

In [ ]:
wordCountDF.printSchema()
wordCountDF.show()

In [ ]:
topNDF = (
    wordCountDF
    .orderBy(sf.col("cnt").desc())
    .limit(topn)
)


In [ ]:
topNDF.printSchema()
topNDF.show()

In [ ]:
# После всех экспериментов собираем итоговый "чистый" код:
docsDF = spark.read.text(raw_text_path)

wordCountDF = (
    docsDF
    .select(
        sf.explode(
            sf.split(
                sf.regexp_replace(
                    sf.lower(sf.col("value")), 
                    "([^a-z ]+)",
                    ""),
                " ")
        ).alias("word"))
    .where(sf.col("word") != "")
    .groupBy(sf.col("word"))
    .agg(sf.count("word").alias("cnt"))
)

topNDF = (
    wordCountDF
    .orderBy(sf.col("cnt").desc())
    .limit(topn)
)

topNDF.show()

In [ ]:
# После работы обязательно отключаем спарк и отдаем ресурсы!
spark.stop()